In [78]:
#1 import lib yg dibutuhkan
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
import easyocr
import os
from IPython.display import display
from ipywidgets import FileUpload, Button
from PIL import Image

#2 buat variabel folder buat nanti simpan gambar sekalian buat foldernya kalo belum ada yaa
OUTPUT_FOLDER = "output/"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

#3 aku buat fungsi dulu nih, buat preproses gambar, deteksi obyek pake YOLO, sama deteksi tulisan pake OCR
def proses_img(cImPath):
    cImg = cv2.imread(cImPath)
    cOutputImg = cImg.copy()

    if cOutputImg is None:
        print("ERROR: Gambar tidak terbaca.")
    else:
        he, wi, ch = cOutputImg.shape
        
        #3.1 Preprocessing
        print("\n=== STEP 1: Preprocessing ===")
        # buat jadi grey dulu biar lebih stabil dan mudah saat pemrosesan selanjutnya
        gray  = cv2.cvtColor(cImg, cv2.COLOR_BGR2GRAY)
        blur  = cv2.GaussianBlur(gray, (5,5), 0)
        edges = cv2.Canny(blur, 100, 200)
        
        #Simpan hasil preprocessing ke folder output
        cv2.imwrite(os.path.join(OUTPUT_FOLDER, "gray.jpg"), gray)
        cv2.imwrite(os.path.join(OUTPUT_FOLDER, "blur.jpg"), blur)
        cv2.imwrite(os.path.join(OUTPUT_FOLDER, "edges.jpg"), edges)
    
        plt.figure(figsize=(12,4))
        plt.subplot(1,3,1); plt.imshow(gray, cmap='gray') 
        plt.title("Gray")
        plt.subplot(1,3,2); plt.imshow(blur, cmap='gray') 
        plt.title("Blur")
        plt.subplot(1,3,3); plt.imshow(edges, cmap='gray')
        plt.title("Edges")
        plt.show()
    
        #3.2 YOLOv8 deteksi objeknya
        print("\n=== STEP 2: YOLOv8 Mendeteksi Obyek ===")
        model      = YOLO("yolov8n.pt")
        #deteksi obyek
        hasilDeteksi = model(cImPath)
        print(f"hasil deteksi : {hasilDeteksi}")
        
        # gambar box di cOutputImg
        for vaBox in hasilDeteksi[0].boxes:
            x1, y1, x2, y2 = map(int, vaBox.xyxy[0])
            nConf = vaBox.conf[0]
            nCls = int(vaBox.cls[0])
            cLabel = f"{nCls}: {nConf:.2f}"
            # memvisualisasi data dengan openCV
            cv2.rectangle(cOutputImg, (x1, y1), (x2, y2), (0,255,0), 2)
            cv2.putText(cOutputImg, cLabel, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

        plt.figure(figsize=(12,4))
        plt.subplot(1,3,1)
        plt.imshow(cOutputImg, cmap='gray') 
        plt.title("Output YOLO")
        cv2.imwrite(os.path.join(OUTPUT_FOLDER, "YOLO.jpg"), cOutputImg)
        
        #3.3 Baca tulisannya dengan OCR
        print("\n=== STEP 3: OCR ===")
        #buat canvar baru nanti untuk gambar tulisannya aja
        cCanvasImg = np.zeros((he, wi, ch), dtype=np.uint8)
        
        reader = easyocr.Reader(['id'])
        vaHasilRead = reader.readtext(cImPath)
        for hasilRead in vaHasilRead:
            vabbox, cText, nConf = hasilRead
            (top_left, top_right, bottom_right, bottom_left) = vabbox
            
            top_left     = tuple(map(int, top_left))
            bottom_right = tuple(map(int, bottom_right))
            text_pos  = (top_left[0], top_left[1] - 5)
            
            # visualisasi box dan tulisannya pada gambar
            cv2.rectangle(cOutputImg, top_left, bottom_right, (255,0,0), 2)
            cv2.putText(cOutputImg, f"{cText} ({nConf:.2f})", (top_left[0], top_right[1]-10),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)
            cv2.putText(cCanvasImg, f"{cText}", text_pos, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            print(f"Text: {cText}, Confidence: {nConf:.2f}")
    
        #3.4 tampilkan gambar setelah YOLO & OCR dan simpan
        cPathAkhir = os.path.join(OUTPUT_FOLDER, "final_output.jpg")
        cPathAkhir2 = os.path.join(OUTPUT_FOLDER, "final_output_2.jpg")
        cv2.imwrite(cPathAkhir, cOutputImg)
        cv2.imwrite(cPathAkhir2, cCanvasImg)
    
        plt.figure(figsize=(8,6))
        plt.imshow(cv2.cvtColor(cOutputImg, cv2.COLOR_BGR2RGB))
        plt.title("Final Output (YOLO + OCR)")
        plt.axis('off')
        plt.show()
        
        plt.figure(figsize=(8,6))
        plt.imshow(cv2.cvtColor(cCanvasImg, cv2.COLOR_BGR2RGB))
        plt.title("Tulisan Final Output (YOLO + OCR)")
        plt.axis('off')
        plt.show()
        
        print(f"\nGambar final disimpan : {cPathAkhir}")

In [79]:
#4 buat widget pembantu buat upload
wUpload = FileUpload(
    accept='image/jpg',
    multiple=False
)

#5 buat jaga jaga aku buat juga btn untuk trigger fungsi proses
wButton = Button(description="Proses Gambar")
display(wUpload)
display(wButton)

#6 ngesimpan gambar yg diupload
def simpan_file_upl(chg):
    print(f"Gambar diproses... {chg}")

    if len(wUpload.value) > 0:
        vaFile     = wUpload.value[0]   # ambil tuple pertama
        cName      = vaFile['name']     # nama file
        vaContent  = vaFile['content']  # isi file (bytes)
        
        # simpan file
        cImPath = os.path.join(OUTPUT_FOLDER, cName)
        with open(cImPath, "wb") as f:
            f.write(vaContent)
        
        print(f"Gambar disimpan : {cImPath}")
        # panggil process_image
        proses_img(cImPath)
    else:
        print("Upload dulu gambarnya!")

#7 fungsi buat jaga jaga kalo gk deteksi otomatis file yg diupload
def btn_onclick(b):
    print(f"Gambar diproses... {b}")

    if len(wUpload.value) > 0:
        vaFile     = wUpload.value[0]   # ambil tuple pertama
        cName      = vaFile['name']     # nama file
        vaContent  = vaFile['content']  # isi file (bytes)
        
        # simpan file
        cImPath = os.path.join(OUTPUT_FOLDER, cName)
        with open(cImPath, "wb") as f:
            f.write(vaContent)
        
        print(f"Gambar disimpan : {cImPath}")
        # panggil process_image
        proses_img(cImPath)
    else:
        print("Upload dulu gambarnya!")


wUpload.observe(simpan_file_upl, names='value')
wButton.on_click(btn_onclick)

FileUpload(value=(), accept='image/jpg', description='Upload')

Button(description='Proses Gambar', style=ButtonStyle())